In [11]:
import numpy as np
import pandas as pd
import random as rd
from collections import defaultdict
from sklearn import linear_model, datasets
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mutual_info_score
import entropy_estimators as ee
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv('train.csv')
train = train.set_index('ID')

/Users/adam/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [58]:
samp = train.loc[rd.sample(train.index, 5000)]

In [59]:
# Variable types
vartypes = train.dtypes
vartypes

cat_df = train.loc[:, vartypes[vartypes=='O'].index]
#int_df = train.loc[:, vartypes[vartypes=='int64'].index]
#float_df = train.loc[:, vartypes[vartypes=='float64'].index]
num_df=train.loc[:, vartypes[vartypes=='float64'].index | vartypes[vartypes=='int64'].index]

In [32]:
# Variable types
vartypes = train.dtypes
vartypes.value_counts()

int64      1405
float64     477
object       51
dtype: int64

In [60]:
#Remove features of int_df that only have a single value
numun=num_df.apply(lambda x: x.nunique())
#Determine which columns have only one entry
ii = np.where(numun == 1)[0]

#Remove Columns with only one variable
num_df.drop(num_df.columns[ii], axis=1, inplace=True)

In [12]:
#Now we will Impute float_df, replacing all ``null'' with the...mean?  No-use most frequent.  mean will give you 
#decimals which don't make sense here.  Ok-this didn't make any difference. 
#Warning - takes about 8 minutes
imp=preprocessing.Imputer(strategy='mean');
num_df=imp.fit_transform(num_df)

In [26]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(num_df[:,0:1839],train['target'],test_size=0.33)

In [29]:
RForest=RandomForestClassifier(n_estimators=75)
model=RForest.fit(Xtrain,Ytrain)
predtrain = RForest.predict_proba(Xtrain)
predtest = RForest.predict_proba(Xtest)
print roc_auc_score(Ytrain,predtrain[:,1])
print roc_auc_score(Ytest,predtest[:,1])

1.0
0.75776185301


In [423]:
#Step 1: Run through data, change everything that a) is much bigger than median and 
#b)has many values to NaN

for ii in range(0,1):
#for ii in range(0,int_df.shape[1]-1):
    #Determine what number/numbers to replace
    Vls=sort(pd.unique(int_df.iloc[:,ii]))
    #NOT DOING WHAT YOU THINK IT IS!  Vct=int_df.iloc[:,ii].value_counts(ascending=True)
    #lets look at all values 10*med, add if they appear more than 500 times.
    med=median(Vls)
    todel=[]
    for jj in range(0,len(Vct)-1):
        #lets get rid of everything that appears, say, more than 500 times
        if Vct[jj]>500:
            #lets also require that it be, say, 10 times bigger than the median value
            if 10*Vls[jj]>med:
                print Vls[jj]
                todel.append(Vls[jj])
                
    #find all locations where array == an element of todel
    #set all these locations to NaN
    for kk in range(0,len(todel)-1):
        todelloc = np.where(int_df.iloc[:,ii] == todel[kk])[0]
        int_df.iloc[todelloc,ii]=float('NaN');
        
        

1
2
3


KeyError: 10

In [445]:
#pd.unique(int_df.iloc[:,4])
#999: 0, 1 note: 99 not  bigger
#99: 3 4 note: 99 much bigger
# for line 26 we have both 9999 and 9996.  line 45 has 9998 and 9999
#Line 120 has 999999996, 999999998, 999999999
#some lines have negative -99999 or something
#If much bigger than median, has many values
print int_df.iloc[:,3].value_counts();
#todelloc = np.where(int_df.iloc[:,3] == 1)[0]
#print int_df.iloc[todelloc,3]

0     133180
1       7897
2       2626
3        673
99       601
4        167
5         51
6         16
7         12
8          4
11         2
18         1
9          1
dtype: int64


In [451]:
gs=0;
k=0
for b in range(0,145230):
    if math.isnan(float_df.iloc[b,k]):
        print b
        gs=gs+1;
        
    

print gs
print float_df.iloc[:,k].isnull().sum()
print float(gs)/float_df.iloc[:,k].isnull().sum()

9855
11910
13018
13447
16524
17461
20511
25658
27429
33001
36093
36534
40061
40317
40848
41185
43410
43629
45875
55156
56206
57191
58586
60229
62100
62455
65682
66061
69329
70000
74293
79003
79268
84358
85066
87829
88207
91939
94335
94705
94729
96148
97962
105196
112029
115558
117069
122319
122527
124410
133942
135973
138754
139295
143274
143915
56
56
1.0


In [8]:
float_df[9850:9860,0]

array([ 2.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.])

In [460]:
float_df.iloc[9850:9860,0]

ID
19545     2
19547     0
19549     0
19551     0
19553     2
19557   NaN
19559     0
19561     0
19566     0
19567     0
Name: VAR_0006, dtype: float64

In [29]:
print full_df.shape
print int_df.shape
print float_df.shape

(290462, 1841)
(145231, 1403)
(145231, 441)


In [27]:
full_df[:,1840-447]

array([ 997.        ,    0.        ,    0.        , ...,  138.45199028,
        138.45199028,  138.45199028])

In [18]:
num_df.shape

(145231, 1841)

In [105]:
Vls=pd.unique(num_df.iloc[:,966])
todelloc = np.where(num_df.iloc[:,1003] == Vls[5])[0]

In [107]:
print Vls

print todelloc[5]
print median(Vls)

[ 0  3  4  2  1 99  5  7 12  8  6  9 10 13 11 14 15 19]
63610
8.5


In [108]:
#Step 1: Run through data, change everything that a) is much bigger than median and 
#b)has many values to NaN

for ii in range(966,967):
#for ii in range(0,int_df.shape[1]-1):
    #Determine what number/numbers to replace
    Vls=pd.unique(num_df.iloc[:,ii])
    for jj in range(0,len(Vls)-1):
        if Vls[jj]>10*median(Vls):
            todelloc = np.where(num_df.iloc[:,ii] == Vls[jj])[0]
            if len(todelloc)>500:
                num_df.iloc[todelloc,ii]=float('NaN');


In [110]:
Vls=pd.unique(num_df.iloc[:,966])
#todelloc = np.where(num_df.iloc[:,1003] == Vls[5])[0]
print Vls

[  0.   3.   4.   2.   1.  nan   5.   7.  12.   8.   6.   9.  10.  13.  11.
  14.  15.  19.]
